In [ ]:
import dpp_nets.dpp as dpp

In [ ]:
dpp.sample_dpp

In [ ]:
import numpy
A = numpy.random.rand(10,10)
L = A.dot(A.T)
e, v = numpy.linalg.eig(L)
subset = dpp.sample_dpp(e, v, one_hot=True)

In [ ]:
subset

In [ ]:
np.zeros(3).dtype

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Function
from torch.autograd import Variable

import dpp_nets.dpp as dpp

class DPP(Function):
    """
    Uses Numpy Functions to sample from the DPP implicitly 
    defined through embd, returns score as a gradient in the
    backward computation (needs to be complemented by hooks
    for REINFORCE or control variate training)

    Arguments:
    Depending on whether you're training Double or Float, provide
    dtype = torch.FloatTensor
    dtype = torch.DoubleTensor
    """
    def __init__(self, dtype):
        self.dtype = dtype

    def forward(self, embd):

        # Perform SVD to get eigenvalue decomposition of L
        u1, s, u2 = torch.svd(embd)
        e = torch.pow(s,2).numpy()
        v = u1.numpy()

        # Sample subset from the DPP
        subset = torch.from_numpy(dpp.sample_dpp(e, v, one_hot=True))
        subset = subset.type(self.dtype)
        # Save tensors for backward (gradient computation)
        self.save_for_backward(embd, subset)

        return subset
        
    def backward(self, grad_output):
        embd, subset = self.saved_tensors
        embd, subset = embd.numpy(), subset.numpy()
        score = torch.from_numpy(dpp.score_dpp(embd, subset))

        return score.type(self.dtype)

In [6]:
import dpp_nets.my_torch as my_torch
import torch
from torch.autograd import Variable

dtype = torch.FloatTensor
#dtype = torch.DoubleTensor
my_layer = my_torch.DPPLayer(dtype)
A = Variable(torch.randn(10,10), requires_grad=True)
subset = my_layer(A)
l = torch.sum(subset)
l.backward()
print(type(subset.data))
print(type(A.grad.data))

<class 'torch.FloatTensor'>
<class 'torch.FloatTensor'>


In [34]:
A = torch.randn(10,5)

In [38]:
torch.stack(A.chunk(2,0))


(0 ,.,.) = 
  0.3213 -0.1642 -1.5809  1.2290 -0.3483
 -0.3959 -0.7861  0.1409 -0.0809  0.1390
  1.4156 -1.3082  1.4605  1.3381 -2.8320
  1.0430 -0.2916  1.1915  0.0912 -0.0003
  0.9861 -0.2945 -0.8929 -2.6913  0.4312

(1 ,.,.) = 
  0.5129  1.1571 -0.3850  0.2668  0.5106
 -1.1796  0.5415 -1.9097  0.7965 -1.6713
  1.9068  0.6041  0.5500  0.5249 -0.1018
  0.0731  0.6370 -0.1749  0.0695 -1.0927
 -0.3434 -0.6750 -1.2442  1.3104 -2.1397
[torch.FloatTensor of size 2x5x5]

In [26]:
import torch.nn as nn

In [28]:
my_layer = nn.Linear(10,20)

In [32]:
my_layer.type(torch.FloatTensor)

Linear (10 -> 20)

In [33]:
type(my_layer.weight.data)

torch.FloatTensor

In [39]:
import numpy as np

In [42]:
np.random.choice(20,1) + 1

array([8])

In [43]:
np.random.choice(30, 8)

array([18, 28,  1, 26, 14, 29, 25, 14])

In [58]:


GLUE = 5
SIGNAL = 50
set_size = 40
pred_in = 150
words = np.random.randn(set_size, pred_in)

# Sample a number of clusters (between 1 and 20) present in training instance
n_clusters = 1 + np.random.choice(20,1) 
# Will repeat cluster indices to fill upto set_size
rep = (set_size // n_clusters) + 1 

# Sample cluster indices 
clusters = np.random.choice(pred_in // GLUE, n_clusters, replace=False)

# Find column indices associated with cluster indices
col_idx = np.array([np.arange(i*GLUE, i*GLUE + GLUE) for i in clusters]).flatten()
# Repeat indices to fill upto set_size 
col_idx = np.tile(col_idx, rep)[:(set_size * GLUE)]

# Overwrite training data with signal according to column indices
words[np.repeat(np.arange(40), GLUE), col_idx] = np.random.normal(SIGNAL, 1, (set_size * GLUE))

# Create context 
context = np.tile(np.sum(words, axis=0), (set_size, 1))

In [67]:
np.tile(np.sum(words, axis = 0), 10).shape

(1500,)

In [55]:
dtype

torch.FloatTensor

In [72]:
A = Variable(torch.randn(10,10))

In [73]:
torch.sum(A)

Variable containing:
 11.1885
[torch.FloatTensor of size 1]

In [76]:
A.data.sum()

11.188507980899885